# Day 11: Dumbo Octopus
There are 100 octopuses arranged neatly in a 10 by 10 grid. Each octopus slowly gains energy over time and flashes brightly for a moment when its energy is full.
The energy level of each octopus is a value between 0 and 9.
During a single step, the following occurs:

First, the energy level of each octopus increases by 1.
Then, any octopus with an energy level greater than 9 flashes. This increases the energy level of all adjacent octopuses by 1, including octopuses that are diagonally adjacent. If this causes an octopus to have an energy level greater than 9, it also flashes. This process continues as long as new octopuses keep having their energy level increased beyond 9. (An octopus can only flash at most once per step.)
Finally, any octopus that flashed during this step has its energy level set to 0, as it used all of its energy to flash.
Adjacent flashes can cause an octopus to flash on a step even if it begins that step with very little energy.
How many total flashes are there after 100 steps?

In [ ]:
/// Clamps a given input to a maximum
let inline ceilingAt max i = if i > max then max else i
/// Clamps a given input to a minimum
let inline floorAt min i = if i < min then min else i
/// Clamps a value between a min and max
let inline clampBetween min max i = i |> floorAt min |> ceilingAt max

/// Expands a string of numbers into a sequence of floats less than 1.0
let inline toFloats (str:string) =
    let rec decompose (buff) (l:Int64)  =
        let b = seq { yield l % 10L |> decimal |> (fun fl -> fl / 10m); yield! buff }
        if l < 10 then b
        else decompose b (l / 10L)

    str |> int64 |> decompose Seq.empty

/// Finds the neighboring elements of a jagged array and returns them (exclusive of the start coord)
let getNeighborsFor (x:int)(y:int) grid = seq {
    for a in [|floorAt 0 (y-1)..ceilingAt 9 (y+1)|] do
        for b in [|floorAt 0 (x-1)..ceilingAt 9 (x+1)|] do
            if a <> y || b <> x then yield grid |> Seq.item a |> Seq.item b
}

In [ ]:
//  Generate a clean game board from string input
let generateGameState = System.IO.File.ReadAllLines("../Data/Day11.txt") |> Array.map toFloats

/// Move the game state forward by a single step
let inline takeStep (data:seq<seq<decimal>>) = 
    Seq.mapi (fun y row ->
        Seq.map (fun i -> 
            ceilingAt 1m (i+0.1m)
        ) row
    ) data

let aggBoard = Array2D.create 10 10 0

/// Progresses the game forward by the number of steps starting a given step
let rec takeSteps stop step data =
    // Setup a board to start capturing our flashes    
    let applyNeighbors (state:seq<seq<decimal>>) =
        Seq.mapi (fun y row ->
            Seq.mapi (fun x d ->
                (x,y,state) |||> getNeighborsFor |> Seq.sumBy (fun i -> match i with 1m -> 1m | _ -> 0m)
            ) row
        ) state
    let newState = data |> takeStep 
    let neighborInfluence = newState |> applyNeighbors |> array2D

    if stop = step then data
    else 
        Seq.mapi (fun y row ->
            Seq.mapi (fun x d ->
                let peak = d + (neighborInfluence.[x,y] / 10m) |> ceilingAt 1m
                match peak with
                | 1m ->
                    Array2D.set aggBoard x y (aggBoard.[x,y] + 1)
                    0m
                | p -> p
            ) row
        ) newState
        |> takeSteps stop (step + 1) 

//  Step 1, generate a clean game state to start with
generateGameState 
//  Step 2, Walk the board forward X number of steps
|> takeSteps 20 0

display aggBoard


index,value
0,595.96
1,862.50
2,927.00
3,885.00
4,888.00
5,941.25
6,897.75
7,870.75
8,1054.50
9,582.80


## Thoughts
After applying brute force to 10 iterations of just flashing individuals with no spillover, we can determine that the flash rate for each squid is equal to the number of steps / 10. In other words if a squid were alone, over 120 steps a single squid would flash 12 times and after 120 steps, the squid's energy level would be equal to where it started as 120 is a multiple of 10.

Given the goal is to count the number of flashes, we know the answer with ***at least*** be equal to the number of squids * (steps / 10). I.E. 1 squid over 120 steps would be 12 flashes; 20 squids over 109 steps would be 200 flashes.

We also know that a squid cannot flash more than once each step which means that theoretically, if each squid somehow managed to flash every step, then they'd ***at most*** contribute a total number of flashes equal to the number of steps. I.E. we can use the step counter as max/clamp per step to determine whether a squid has flashed per step.

To solve this, we'll decompose each number in each string into a floating point number less than 1.0. Each step we'll increment by .1 which will allow our whole number to keep the state of our flashes. At the same time we'll clamp our whole number to never be higher than our step. This way when we hit our max steps, we'll just sum our whole numbers for the answer.
> An observation around how the problem is formed: It would appear from the description that once a squid has flashed, their power level wont increase from 0 again during this step despite any influence from neighbors. I.E. it says clearly that they wont flash twice but never calls out that they can't grow to the edge of flashing. I'm assuming from their example that once flashed, they're spent and wont grow strength again until the next step.

The problem is going to be when trying to determine if a neighbor has flashed. We can't just say that they have if their whole number equals the step count because not every on flashes each step though some do.